In [32]:
import pandas as pd
import numpy as np
import sagemaker
from sagemaker.predictor import csv_serializer
import boto3
import re
import os
from sagemaker import get_execution_role

role = get_execution_role()

df=pd.read_csv('s3://sage-maker-3/DadosTeseLogit.csv',sep=',',header=0)
sel=np.where(df.corr()['selected']>.45)[0]
df=df.iloc[:,[30,14,17,19]]
df

,selected,c0_disponib,c0_pontual,c0_atencao
0,1,5,5,5
1,1,5,3,3
2,1,5,5,5
3,1,5,5,5
4,1,5,5,5
5,1,5,5,5
6,1,5,5,5
7,1,4,5,5
8,1,4,4,3
9,1,5,5,5


In [33]:
containers  = {'us-west-2': '433757028032.dkr.ecr.us-west-2.amazonaws.com/xgboost:latest',
              'us-east-1': '811284229777.dkr.ecr.us-east-1.amazonaws.com/xgboost:latest',
              'us-east-2': '825641698319.dkr.ecr.us-east-2.amazonaws.com/xgboost:latest',
              'eu-west-1': '685385470294.dkr.ecr.eu-west-1.amazonaws.com/xgboost:latest',
              'ap-northeast-1': '501404015308.dkr.ecr.ap-northeast-1.amazonaws.com/xgboost:latest'}

In [34]:
bucket =  'sage-maker-3'
prefix = 'sagemaker/xgboost-churn'

In [35]:
df.to_csv('train.csv', header=False, index=False)
boto3.Session().resource('s3').Bucket(bucket).Object(os.path.join(prefix, 'train/train.csv')).upload_file('train.csv')
s3_input_train = sagemaker.s3_input(s3_data='s3://{}/{}/train'.format(bucket, prefix), content_type='csv')

In [36]:

sess =sagemaker.Session()

xgb = sagemaker.estimator.Estimator(containers[boto3.Session().region_name],
                                    role, 
                                    train_instance_count=1, 
                                    train_instance_type='ml.m4.xlarge',
                                    output_path='s3://{}/{}/output'.format(bucket, prefix),
                                    sagemaker_session=sess)
xgb.set_hyperparameters(max_depth=5,
                        eta=0.2,
                        gamma=4,
                        min_child_weight=6,
                        subsample=0.8,
                        silent=0,
                        objective='binary:logistic',
                        num_round=100)

xgb.fit({'train': s3_input_train})

INFO:sagemaker:Creating training-job with name: xgboost-2018-06-01-19-59-20-161


.....................
Arguments: train
[2018-06-01:20:02:40:INFO] Running standalone xgboost training.
[2018-06-01:20:02:40:INFO] Path /opt/ml/input/data/validation does not exist!
[2018-06-01:20:02:40:INFO] File size need to be processed in the node: 0.0mb. Available memory size in the node: 8653.07mb
[2018-06-01:20:02:40:INFO] Deduced delimiter of CSV input is ','
[20:02:40] S3DistributionType set as FullyReplicated
[20:02:41] 98x3 matrix with 294 entries loaded from /opt/ml/input/data/train?format=csv&label_column=0&delimiter=,
[20:02:41] src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 0 extra nodes, 2 pruned nodes, max_depth=0
[0]#011train-error:0.183673
[20:02:41] src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 0 extra nodes, 2 pruned nodes, max_depth=0
[1]#011train-error:0.183673
[20:02:41] src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 0 extra nodes, 2 pruned nodes, max_depth=0
[2]#011train-error:0.183673
[20:02:41] src/tree/updater_prune.cc:74: tree 

===== Job Complete =====
Billable seconds: 96


In [37]:
xgb_predictor = xgb.deploy(initial_instance_count=1,
                           instance_type='ml.m4.xlarge')

INFO:sagemaker:Creating model with name: xgboost-2018-06-01-20-03-21-570
INFO:sagemaker:Creating endpoint with name xgboost-2018-06-01-19-59-20-161


---------------------------------------------------!

In [66]:
xgb_predictor.content_type = 'text/csv'
xgb_predictor.serializer = csv_serializer
xgb_predictor.deserializer = None


In [71]:
df3=df.iloc[:,[1,2,3]]
np.float(xgb_predictor.predict(df3.iloc[0,:]).decode('utf-8'))

0.806817531586

In [72]:
def  predict(data, rows=98):
    split_array = np.array_split(data, int(data.shape[0] / float(rows) + 1))
    predictions = ''
    for array in split_array:
        predictions = ','.join([predictions, xgb_predictor.predict(array).decode('utf-8')])

    return np.fromstring(predictions[1:], sep=',')

predictions = predict(df3.as_matrix())
predictions

array([0.80681753, 0.80681753, 0.80681753, 0.80681753, 0.80681753,
       0.80681753, 0.80681753, 0.80681753, 0.80681753, 0.80681753,
       0.80681753, 0.80681753, 0.80681753, 0.80681753, 0.80681753,
       0.80681753, 0.80681753, 0.80681753, 0.80681753, 0.80681753,
       0.80681753, 0.80681753, 0.80681753, 0.80681753, 0.80681753,
       0.80681753, 0.80681753, 0.80681753, 0.80681753, 0.80681753,
       0.80681753, 0.80681753, 0.80681753, 0.80681753, 0.80681753,
       0.80681753, 0.80681753, 0.80681753, 0.80681753, 0.80681753,
       0.80681753, 0.80681753, 0.80681753, 0.80681753, 0.80681753,
       0.80681753, 0.80681753, 0.80681753, 0.80681753, 0.80681753,
       0.80681753, 0.80681753, 0.80681753, 0.80681753, 0.80681753,
       0.80681753, 0.80681753, 0.80681753, 0.80681753, 0.80681753,
       0.80681753, 0.80681753, 0.80681753, 0.80681753, 0.80681753,
       0.80681753, 0.80681753, 0.80681753, 0.80681753, 0.80681753,
       0.80681753, 0.80681753, 0.80681753, 0.80681753, 0.80681